1. 문서의 내용을 읽어옴
2. 문서를 쪼갬
    - Token 수 초과로 답변 생성을 못하거나
    - Input 이 길면 답변 생성이 오래걸림
3. Embedding -> Vector DB에 저장
4. 질문이 있을 때, Vector DB에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달함

In [2]:
from calendar import prmonth

from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)


loader = UnstructuredWordDocumentLoader('./data/tax-law.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [4]:
len(document_list)

193

In [5]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

embedding = OpenAIEmbeddings(
    model = 'text-embedding-3-large'
)

In [6]:
from langchain_chroma import Chroma

# Chroma는 인메모리 DB이므로 종료 시 데이터가 휘발됨
# persist_directory를 통해 영속성을 유지할 수 있음
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name= 'chroma-tax',persist_directory="./chroma")

In [26]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

retrieved_docs = database.similarity_search(query, k=3)

In [27]:
retrieved_docs

[Document(id='c6070b0f-28e6-45ea-a3f2-e58b2c974b8d', metadata={'source': './data/tax-law.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 \n\n\n\n변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31., 2025. 10. 1.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초

In [25]:
prompt = f"""
[Identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')


In [30]:
ai_message = llm.invoke(prompt)

In [31]:
ai_message

AIMessage(content="연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 우선 과세표준을 산정하고, 여기에 소득세율을 적용해야 합니다. 이 과정은 다음과 같은 일반적인 단계로 진행됩니다.\n\n1. **총급여에서 필요경비 및 공제항목 차감:**\n   - 연봉에서 국민연금, 건강보험, 고용보험 등 의무 납부 항목과 기본공제 및 추가 공제를 차감하여 과세표준을 산정합니다.\n\n2. **과세표준에 따라 소득세율 적용:**\n   - 한국의 소득세는 과세표준에 따라 여러 세율 구간이 설정되어 있습니다. \n   - 예를 들면, 소득세율은 대체로 다음과 같이 구성됩니다.\n     - 1,200만원 이하: 6%\n     - 1,200만원 초과 ~ 4,600만원 이하: 15%\n     - 4,600만원 초과 ~ 8,800만원 이하: 24%\n     - 그 이상 구간은 점진적으로 높아집니다.\n\n연봉 5천만원의 경우, 이를 과세표준으로 환산하고 적합한 세율을 적용하면 소득세가 산정됩니다.\n\n3. **세액 공제:**\n   - 연말정산 시 다양한 세액 공제항목을 적용할 수 있습니다. 예를 들어 보험료 세액공제, 의료비, 교육비 및 기부금 공제 등이 있습니다.\n\n구체적인 계산은 각 개인의 공제 항목에 따라 달라지므로 정확한 소득세를 산정하기 위해서는 개인의 공제 내역과 과세 대상을 명확히 해야 합니다. 따라서, 보다 정확한 세금 계산은 국세청의 '연말정산 간소화 서비스'나 세무사를 통해 확인하는 것이 좋습니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 411, 'prompt_tokens': 2990, 'total_tokens': 3401, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 

In [35]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("rlm/rag-prompt", include_model=True)

In [36]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [37]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

combine_docs_chain = create_stuff_documents_chain(
    llm, prompt
)

rag_chain = create_retrieval_chain(
    database.as_retriever(),
    combine_docs_chain
)

rag_chain.invoke({
    "input": query,
    "question": query
})

{'input': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'question': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'context': [Document(id='c6070b0f-28e6-45ea-a3f2-e58b2c974b8d', metadata={'source': './data/tax-law.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 \n\n\n\n변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31., 2025. 10. 1.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당